# Titanic

Dataset is available at Kaggle :[https://www.kaggle.com/c/titanic](https://www.kaggle.com/c/titanic)

**Problem** : In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive

Dataset Desciption and information:

survival  :	Survival    :	0 = No, 1 = Yes
pclass	  :Ticket class	: 1 = 1st, 2 = 2nd, 3 = 3rd
sex	      :  Sex	
Age	      :Age in years	
sibsp	  :# of siblings / spouses aboard the Titanic	
parch	  :# of parents / children aboard the Titanic	
ticket	  : Ticket number	
fare	  :Passenger fare	
cabin	  :Cabin number	
embarked  : Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

To work on this problem ,we can start with importing below libraries for Analysis

1. Numpy
2. pands
3. matplotlib
4. seaborn
5. warnings
6. re
7. scipy

let's get started

In [1]:
#import Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import scipy
import warnings

#Import Sklearn libary
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
#from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
# With ignore we can avoid warnings generated during execution
warnings.filterwarnings("ignore")

**EDA**

1. Load the 'train_titanic' training and 'test_titanic' dataset
2. store the values of passengerId which we are using later for prediction
3. head() is displaying the first few rows of the dataset
4. now just take overlook of the dataset
5. shape() gives the size of dataset , training dataset contain (891, 12) and testing dataset contain (418, 11) rows and columns
6. dataset contain mixed type of datatype some are objects and some are float

In [3]:
pd.options.display.max_columns = None
dt_train=pd.read_csv("train_titanic.csv",sep=',')
dt_test=pd.read_csv("test_titanic.csv",sep=',')
PassengerId=dt_test['PassengerId']
dt_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print(dt_train.shape)
print(dt_test.shape)

(891, 12)
(418, 11)


In [5]:
print(dt_train.dtypes)


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


**Missing Value Handling**

1. Lets work on missing value in dataset ,with isnull(),we can find missing values in both the dataset
2. with below results it's look some of the columns have missing value
3. Feauture Age, cabin, Embarked, fare has missing value, now lets handle those missing value

In [6]:
dt_copy=dt_train.copy()

In [7]:
print(dt_copy.isnull().sum())
print(dt_test.isnull().sum())
size=len(dt_copy)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [8]:
class data_clean():
    
    
    def __init__(self):
        '''This class contain method require for creating new columns, handling missing value, drop unwanted columns,
        creating dummy variable
        and checking skewness'''
        print("Object creation for data cleanig")
        
    
    def create_new_col(self,data):
        
        '''In dataset, there is name of Passanger is mentioned but we dont require name of passanger
        we can catogerise person using its title name ,this method is used to split name and exctract only 
        title of person e.g Mr, Mrs,Miss
        
        Also in dataset tjere are columns which gives information about passanger and which him how many persons has come
        with this we can create new feature call Family and add all family members in it'''
        
        data['Title'] = data['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
        data['Family']=data['SibSp']+data['Parch']+1
        return data
        
        
    def handlling_missing_val(self,data):
        
        '''This method is usefull for handling missing values in datset, here we are going to replace missing value 
        wirh Age with mean 28 and Embrances with most occuring category S 
        
        Also here we Add we add common for all other title value which are rare'''
        
        value={'Age':28,'Embarked':'S'}
        data.fillna(value=value,inplace=True)
        replace_values = {'Capt':'common','Col':'common', 'Don':'common','Dr':'common','Jonkheer':'common','Lady':'common','Lady':'common',
                 'Major':'common','Mlle':'common','mme':'common','Ms':'common','Rev':'common','Sir':'common','the Countess':'common'}
        data = data.replace({"Title": replace_values})
        return data
    
    def drop_col(self,data):
        
        '''Below listed columns are not necessary and didnt add any value important and not require for training datset
        so we are dropping it from dataset'''
        
        data.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Fare'],axis=1,inplace=True)
        return data
    
    def dummy(self,data):
        
        '''This method is usefull for converting categorical feature into numerical form'''
        
        data=pd.get_dummies(data,drop_first=True).astype('float')
        return data
    
    def check_skewness(self,data):
        
        '''This feature is used for, if the skewness of numerical feature is greater than 0.75 then we will convert that feature
        into logarthmic form'''
        
        col_number = data.select_dtypes(include=['number']).columns.tolist()
        print(col_number)
        for col_transform in col_number:
            skewness = scipy.stats.skew(data[col_number])
            print(skewness)
        if skewness.all() > 0.75:
            data[col_number] = np.log1p(data[col_number])
        return data

In [9]:
#Preprocess And MLmodelling
class PreProcessing():
    
    def __init__(self):
        
        '''This class is usefull for Preprocessing and finding the best Algorithm to train model'''
        
        print("Preprocessing object has created")
    
    def default_best_algo(self,X_train,Y_train):
        
        '''This method is usefull for finding the default best Algorithm for training the dataset
        it evaluate Algorith by degault and train with KFOLD and cross_val_score'''
        model=[]

        model.append(('SVC',SVC()))
        model.append(('NB',GaussianNB()))
        model.append(('DC',DecisionTreeClassifier()))
        model.append(('KNN',KNeighborsClassifier()))
        model.append(('LogR',LogisticRegression()))
        
        names=[]
        result=[]
        scoring='accuracy'
        for name , algo in model:
            kfold=KFold(n_splits=10, random_state=7)
            cv_result=cross_val_score(algo,X_train,Y_train,scoring=scoring,cv=kfold)
            result.append(cv_result)
            names.append(name)
            msg=print(name,cv_result.mean()*100,cv_result.std())
    
    
    def best_algo_with_preprocess(self,X_train,Y_train):
        
        '''This method is usefull for training model with Pipeline with adding preprocessing the data
        and them apply ML algo on it'''
        
        model2=[]
        model2.append(('ScaleLogR',Pipeline([('Scaler',StandardScaler()),('LogR',LogisticRegression())])))
        model2.append(('ScaleSVC',Pipeline([('Scaler',StandardScaler()),('SVC',SVC())])))
        model2.append(('ScaleNB',Pipeline([('Scaler',StandardScaler()),('NB',GaussianNB())])))
        model2.append(('ScaleDC',Pipeline([('Scaler',StandardScaler()),('DC',DecisionTreeClassifier())])))
        model2.append(('ScaleKNN',Pipeline([('Scaler',StandardScaler()),('KNN',KNeighborsClassifier())])))


        names=[]
        result=[]
        for name , algo in model2:
            kfold=KFold(n_splits=10, random_state=7)
            cv_result=cross_val_score(algo,X_train,Y_train,scoring='accuracy',cv=kfold,verbose=0)
            result.append(cv_result)
            names.append(name)
            msg=print(name,cv_result.mean()*100,cv_result.std())
    
    def algo_ensemble(self,X_train,Y_train):
        
        '''This method is usefull for training dataset on Esembles'''
        pipeEse=[]
        pipeEse.append(('ScaledAdaB',Pipeline([('Scaler',StandardScaler()),('AdaB',AdaBoostClassifier())])))
        pipeEse.append(('ScaledGB',Pipeline([('Scaler',StandardScaler()),('GB',GradientBoostingClassifier())])))
        pipeEse.append(('ScaledRFR',Pipeline([('Scaler',StandardScaler()),('RFR',RandomForestClassifier())])))
        pipeEse.append(('ScaledETR',Pipeline([('Scaler',StandardScaler()),('ETR',ExtraTreesClassifier())])))

        names=[]
        result=[]
        for name,model in pipeEse:
            kfolds=KFold(n_splits=10,random_state=7,)
            cv_results=cross_val_score(model,X_train,Y_train,cv=kfolds,scoring='accuracy')
            result.append(cv_results)
            names.append(name)
            print(name, cv_results.mean() ,cv_results.std())

In [10]:
#Final Model
class finalModel():
    
    def __init__(self):
        
        '''This class is create for training model and predicitng the results on test data'''
        print("intialize the model")
    
    def model_F(self):
        
        '''this method is usefull training the data and check accuracy'''
        
        scale=StandardScaler().fit(X_train)
        trnsF=scale.transform(X_train)
        modelB=SVC(C=1.0,kernel='rbf',random_state=7)
        modelB.fit(trnsF,Y_train)
        trnsF2=scale.transform(X_test)
        decision=modelB.predict(trnsF2)
        finalScore=accuracy_score(decision,Y_test)
        finalScore1=confusion_matrix(decision,Y_test)
        finalScore2=classification_report(decision,Y_test)
        print(finalScore)
        print(finalScore1)
        print(finalScore2)
        
    def model_Predict(self,x):
        
        '''This method is usefull for actual predicitng the test data and check accuracy'''
        scale=StandardScaler().fit(X_train)
        trnsF=scale.transform(X_train)
        modelB=SVC(C=1.0,kernel='rbf',random_state=7)
        modelB.fit(trnsF,Y_train)
        trnsF2=scale.transform(X_test)
        trnsF3=scale.transform(x)
        #decision=modelB.predict(trnsF2)
        decision1=modelB.predict(trnsF3)
        #test_Survived = pd.Series(decision1, name="Survived")
        test_Survived = pd.Series(decision1, name="Survived")
        Submission3 = pd.concat([PassengerId,test_Survived],axis=1)
        
        return Submission3
    

In [11]:
# create object of class data_clean
dataclean=data_clean()
dataclean_test=data_clean()

# call method create_new_col and apply is on both the dataset
dt_copy=dataclean.create_new_col(dt_copy)
dt_test=dataclean_test.create_new_col(dt_test)

#call method handling missing_val and appy on both the dataset
dt_copy=dataclean.handlling_missing_val(dt_copy)
dt_test=dataclean_test.handlling_missing_val(dt_test)

#Call drop_col method and apply it on both the dataset
dt_copy=dataclean.drop_col(dt_copy)
dt_test=dataclean_test.drop_col(dt_test)

#call dummy method and apply it on both the dataset
dt_copy=dataclean.dummy(dt_copy)
dt_test=dataclean_test.dummy(dt_test)

# call check_skewness method and apply it on both the dataset
dt_copy=dataclean.check_skewness(dt_copy)
dt_test=dataclean_test.check_skewness(dt_test)

dt_copy
dt_test


Object creation for data cleanig
Object creation for data cleanig
['Survived', 'Pclass', 'Age', 'Family', 'Sex_male', 'Embarked_Q', 'Embarked_S', 'Title_Miss', 'Title_Mme', 'Title_Mr', 'Title_Mrs', 'Title_common']
[ 0.47771747 -0.62948588  0.50938526  2.72284767 -0.61787841  2.94381099
 -1.00796412  1.46707327 29.7993477  -0.32520332  2.07151765  5.59457671]
[ 0.47771747 -0.62948588  0.50938526  2.72284767 -0.61787841  2.94381099
 -1.00796412  1.46707327 29.7993477  -0.32520332  2.07151765  5.59457671]
[ 0.47771747 -0.62948588  0.50938526  2.72284767 -0.61787841  2.94381099
 -1.00796412  1.46707327 29.7993477  -0.32520332  2.07151765  5.59457671]
[ 0.47771747 -0.62948588  0.50938526  2.72284767 -0.61787841  2.94381099
 -1.00796412  1.46707327 29.7993477  -0.32520332  2.07151765  5.59457671]
[ 0.47771747 -0.62948588  0.50938526  2.72284767 -0.61787841  2.94381099
 -1.00796412  1.46707327 29.7993477  -0.32520332  2.07151765  5.59457671]
[ 0.47771747 -0.62948588  0.50938526  2.72284767 -0

,Pclass,Age,Family,Sex_male,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_common
0,1.386294,3.569533,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.693147,0.000000,0.0
1,1.386294,3.871201,1.098612,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.693147,0.0
2,1.098612,4.143135,0.693147,0.693147,0.693147,0.000000,0.000000,0.000000,0.693147,0.000000,0.0
3,1.386294,3.332205,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.0
4,1.386294,3.135494,1.386294,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.693147,0.0
5,1.386294,2.708050,0.693147,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.0
6,1.386294,3.433987,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.000000,0.0
7,1.098612,3.295837,1.386294,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.0
8,1.386294,2.944439,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.0
9,1.386294,3.091042,1.386294,0.693147,0.000000,0.693147,0.000000,0.000000,0.693147,0.000000,0.0


In [12]:
# Dataset splitting
X=dt_copy.drop(['Survived'],axis=1).astype(float)
Y=dt_copy['Survived'].astype(float)

lab_enc =LabelEncoder()
Y = lab_enc.fit_transform(Y)


X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=7,test_size=0.20)

In [13]:
# create object of preprocessing class and apply method default_best_algo on train_test_split dataset
preprocess=PreProcessing()
preprocess.default_best_algo(X_train,Y_train)

Preprocessing object has created
SVC 81.3302034428795 0.0482345498846718
NB 76.8290297339593 0.06197319251776504
DC 80.76682316118935 0.046729880937458494
KNN 80.76682316118935 0.05157303350710465
LogR 81.04655712050078 0.050338024526183814


In [14]:
# apply method best_algo_with_preprocess on train_test_split dataset
preprocess.best_algo_with_preprocess(X_train,Y_train)

ScaleLogR 82.87558685446011 0.06040872281864481
ScaleSVC 83.43896713615024 0.05536237047911203
ScaleNB 70.51643192488261 0.08517124540268133
ScaleDC 80.06651017214396 0.05058467583513834
ScaleKNN 81.75860719874804 0.05696750269261419


In [15]:
# apply method algo_ensemble on train_test_split dataset
preprocess.algo_ensemble(X_train,Y_train)

ScaledAdaB 0.8203834115805947 0.05664047879223724
ScaledGB 0.8203638497652582 0.05312652320844465
ScaledRFR 0.8119913928012519 0.05723577572733047
ScaledETR 0.802112676056338 0.05878780848370306


In [16]:
scaled=StandardScaler().fit(X_train)
trans=scaled.transform(X_train)

C_val=np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.2,1.5,1.8,2.0])
kernal=['linear', 'poly', 'rbf', 'sigmoid']
par_grid=dict(C=C_val,kernel=kernal)
fitmodel=SVC()
kfold=KFold(n_splits=10, random_state=7)
grid=GridSearchCV(estimator=fitmodel,param_grid=par_grid,scoring='accuracy',cv=kfold)
grid_result=grid.fit(trans,Y_train)
print("Best result for gridSearch",grid_result.best_score_,grid_result.best_params_)

Best result for gridSearch 0.8328651685393258 {'C': 1.0, 'kernel': 'rbf'}


In [17]:
model=finalModel()
model.model_F()
Submission3=model.model_Predict(dt_test)



intialize the model
0.776536312849162
[[95 25]
 [15 44]]
              precision    recall  f1-score   support

           0       0.86      0.79      0.83       120
           1       0.64      0.75      0.69        59

   micro avg       0.78      0.78      0.78       179
   macro avg       0.75      0.77      0.76       179
weighted avg       0.79      0.78      0.78       179



In [18]:
Submission3

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0
